In [ ]:
!python3 -V

Python 3.8.15


In [ ]:
!pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install fastai==2.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import gc

In [ ]:
print(torch. __version__)

1.7.1+cu110


In [ ]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/YelpDataset/dataframes

AutoDF.pkl		     HealthMedicalDF.pkl  restaurantDF.pkl
beautyandSpaDF.pkl	     HomeServicesDF.pkl   shoppingDF.pkl
EventPlanningServicesDF.pkl  LocalServicesDF.pkl
foodDF.pkl		     NightlifeDF.pkl


In [ ]:
%cp /content/drive/MyDrive/YelpDataset/dataframes/beautyandSpaDF.pkl /content/

In [ ]:
import pandas as pd
df = pd.read_pickle('beautyandSpaDF.pkl')

In [ ]:
df = df.rename(columns={'text': 'reviewText'})
df = df.rename(columns={'stars': 'overall'})
df = df.rename(columns={'categories': 'category'})

In [ ]:
df = df[['business_id','name','city','state','overall','category','reviewText']]

In [ ]:
df.head(1)

,business_id,name,city,state,overall,category,reviewText
98648,Kq51_lGAgAigqmENITTr-A,Bala Better Health,Bala Cynwyd,PA,4.0,Beauty & Spas,I have nothing but good things to say. Randy and the whole staff was professional and warm. I felt like I was working with people who cared about me and my results and not just $$$. I would highly recommend getting the free evaluation. They will work with you budget wise.


In [ ]:
df['sentenceLength'] = df['reviewText'].str.count(' ') + 1

In [ ]:
df.shape

(282527, 8)

In [ ]:
df.head(2)

,business_id,name,city,state,overall,category,reviewText,sentenceLength
98648,Kq51_lGAgAigqmENITTr-A,Bala Better Health,Bala Cynwyd,PA,4.0,Beauty & Spas,I have nothing but good things to say. Randy and the whole staff was professional and warm. I felt like I was working with people who cared about me and my results and not just $$$. I would highly recommend getting the free evaluation. They will work with you budget wise.,52
98649,t5tBd4p7BA7hNDitBUXxrw,Boise Therapeutic Massage Center,Boise,ID,4.0,Beauty & Spas,"I've been going to Boise Therapeutic Massage for 4 years now. I'm giving it 5 stars because I can always get an appointment the day I call, every therapist is exceptional, and the price is reasonable. I've never been disappointed. It isn't a fancy spa but their clinic is relaxing and the staff makes you feel welcome and I go for the massage and not the spa experience anyway. I've started going to Amy exclusively for all my massages. It's wonderful when you find someone who knows where your trouble spots are and can make you feel human again.",103


In [ ]:
sample_df = df.sample(frac=0.05, random_state=2021).dropna().reset_index()

In [ ]:
sample_df.shape

(14126, 9)

In [ ]:
del df
gc.collect()

19

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [ ]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples of Toys_and_Games_5

In [ ]:
!ls /content/drive/MyDrive/YelpDataset/models

gpt2-finetuned-yelp-reviews-sample-1cycle-Automotive.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-BeautySpa.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Food.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-HomeServices.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Restaurants.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Shopping.pth


In [ ]:
%cp /content/drive/MyDrive/YelpDataset/models/gpt2-finetuned-yelp-reviews-sample-1cycle-BeautySpa.pth /content/

In [ ]:
learn.load("gpt2-finetuned-yelp-reviews-sample-1cycle-BeautySpa").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
"""N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 10
BINS = list(range(10,15))"""
N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 2000
BINS = list(range(10,351))

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split
generated_df_rows = []
for i in range(len(BINS)-1):
    sample_row_df = sample_df[(sample_df["sentenceLength"]>=BINS[i]) & (sample_df["sentenceLength"]<BINS[i+1])]
    bin_proportion = (sample_row_df.shape[0])/float(LEN_SAMPLE_DF)
    n_reviews_to_generate = int((bin_proportion)*N_REVIEWS)+1
    try:
        or_df, cg_df =  train_test_split(sample_row_df, test_size=0.5, random_state=2021)
    except Exception as e:
        print(e)
        continue
    sample_reviews_df = cg_df.sample(n=n_reviews_to_generate,random_state=2021)
    or_sample_reviews_df = or_df.sample(n=n_reviews_to_generate,random_state=2021)
    if BINS[i] in range(10,21):
        max_len = 20
    else:
        max_len = BINS[i]
    for index, row in sample_reviews_df.iterrows():
        text = row["reviewText"]
        prompt = (" ").join(text.strip().split(" ")[:N_INITIAL_WORDS])
        prompt_ids = tokenizer.encode(prompt)
        inp = tensor(prompt_ids)[None].cuda()
        # set seed to reproduce results
        random_seed(seed_value=2021, use_cuda=True)
        preds = learn.model.generate(inp, max_length=max_len, do_sample=True, top_k=0, top_p=0.92, temperature=0.7)
        decoded_preds = tokenizer.decode(preds[0],skip_special_tokens=True)
        generated_df_rows.append([row["category"],row["overall"],decoded_preds,"CG"])
    for index, row in or_sample_reviews_df.iterrows():
        generated_df_rows.append([row["category"],row["overall"],row["reviewText"],"OR"])
#     break
generated_df = pd.DataFrame(generated_df_rows, columns=["category", "rating", "text", "label"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=0, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


In [ ]:
generated_df.shape

(4190, 4)

In [ ]:
generated_df.head()

,category,rating,text,label
0,Beauty & Spas,4.5,Tailors each experience to the point of the fact that they are all very talented. I have,CG
1,Beauty & Spas,4.5,Great place...super friendly atmosphere!! Best pedicure ever! I will definitely be back!,CG
2,Beauty & Spas,3.5,LeAnn does a great job on pedi's! Highly recommend her!,OR
3,Beauty & Spas,5.0,"Great office! Friendly staff, skilled doctors and convenient hours.",OR
4,Beauty & Spas,3.0,Manicure and pedicure....go to Mary Ann! She does an amazing job! She,CG


In [ ]:
def post_process(str, k=4):
    if not str.endswith("."):
        full_stop_split_str = str.split(".")
        clean_full_stop_split_str = [s.strip() for s in full_stop_split_str if s.strip()]
        if len(clean_full_stop_split_str) > 1:
            last_sentence = clean_full_stop_split_str[-1]
        else:
            return str
        last_sentence_len = len(last_sentence.split(" "))
        if last_sentence_len > k:
            return str
        return (". ").join(clean_full_stop_split_str[:-1]).strip()+(".")
    return str

In [ ]:
generated_df['text_'] = generated_df['text'].apply(lambda x: post_process(x))

In [ ]:
generated_df.head()

,category,rating,text,label,text_
0,Beauty & Spas,4.5,Tailors each experience to the point of the fact that they are all very talented. I have,CG,Tailors each experience to the point of the fact that they are all very talented.
1,Beauty & Spas,4.5,Great place...super friendly atmosphere!! Best pedicure ever! I will definitely be back!,CG,Great place...super friendly atmosphere!! Best pedicure ever! I will definitely be back!
2,Beauty & Spas,3.5,LeAnn does a great job on pedi's! Highly recommend her!,OR,LeAnn does a great job on pedi's! Highly recommend her!
3,Beauty & Spas,5.0,"Great office! Friendly staff, skilled doctors and convenient hours.",OR,"Great office! Friendly staff, skilled doctors and convenient hours."
4,Beauty & Spas,3.0,Manicure and pedicure....go to Mary Ann! She does an amazing job! She,CG,Manicure and pedicure....go to Mary Ann! She does an amazing job! She


In [ ]:
generated_df.drop(columns=["text"],axis=1,inplace=True)

In [ ]:
generated_df[1500:1512]

,category,rating,label,text_
1500,Beauty & Spas,4.5,CG,I highly highly recommend! Jen and her team are very professional and professional. I have been to multiple different hair stylists and they are all great. The salon itself is so beautifully decorated and the staff are so friendly. I have been a loyal client for years and will continue to
1501,Beauty & Spas,4.0,CG,I used to go there regularly for the last 10 years. I always have a great experience. The employees are all very friendly. The salon is clean and the staff is very helpful. \n\nThe only reason I can't give it 4 stars is because
1502,Beauty & Spas,4.0,OR,"Go to this place for your pedicure but NOT your nails. If you go in and tell them you are getting both they will rush your pedicure. They have many color options, great pedicure services, however they are not the best at nails. Nice place, nice ppl. Before covid, they served frozen drinks, wine, water, etc."
1503,Beauty & Spas,4.5,OR,"I love DnA Salon! They have wonderful customer service. I have seen 3 stylists because of scheduling and all were amazing! Now I go to Grace every time. I would trust anyone there to cut my hair, which is saying a lot since I used to go home to Boston for haircuts before I found DnA."
1504,Beauty & Spas,3.5,OR,"Travis Jay give better blonde than anyone. I have had at least 25 different stylist and no one has ever listened to me, or taken the time to give me exactly what I want like Travis. And the salon is just gorgeous inside. Overall it was an amazing experience and I highly recommend it to anyone."
1505,Beauty & Spas,4.5,OR,"I've been there twice for piercing and have had a positive experience both times. I was just a walk-in to pierce my nose... all of the guys were friendly, just hanging out in the shop and I was waited on immediately. \n\nI plan to go back for a tattoo.. anyone else have any words of advice?"
1506,Beauty & Spas,4.0,OR,"For them having a late policy, they are constantly running behind on scheduled times. I've waiting the ""9 minutes"" after my appointment time a handful of times. Other then them never being on time, the staff is friendly and knowledgable! Cherry hill and Mount laurel run much better on their schedules than the Cinnaminson location does."
1507,Beauty & Spas,4.5,OR,"My first time here, but very unlikely to be back. Dirty floors. Older, uncomfortable chairs and only a mediocre pedicure. The layout of the place reminds me more of a bus terminal than a spa. Not relaxing at all, and no attempt to reduce the strong headache-inducing acetone smell. Skip it."
1508,Beauty & Spas,4.0,OR,"Love this place! Best consistent hair cuts I have ever had...and they do a great job on my two sons as well. The long waits have gone down since they have basically become appointment only. It's a little more expensive now since they are appointment only, but well worth it in my opinion."
1509,Beauty & Spas,4.5,OR,One of the best salons around! My daughter and I enjoyed a very professional classic manicure during our 'girls day out' and enjoyed the clean friendly atmosphere! A true indulgence and I highly recommend Emily or Melissa or Cindy!! Kathy makes the best hot tea for you to enjoy while you are pampered!


In [ ]:
generated_df.to_csv("Generated_Sentences_BeautySpa.csv", index=None)

In [ ]:
%cp /content/Generated_Sentences_BeautySpa.csv /content/drive/MyDrive/YelpDataset/generatedData